<a href="https://colab.research.google.com/github/thabang-r13/stock-price-prediction/blob/main/TransformerModel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Stock Price Prediction Transformer Model**

In [ ]:
# Import necessary libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from tensorflow import keras
from tensorflow.keras import layers

In [ ]:
# Load the data
df = pd.read_csv('/content/AMZN.csv')

In [ ]:
# Display the first five records
df.head()

In [ ]:
# Extracting the 'Close' column for analysis
closing_price = df['Close']

In [ ]:
# Normalize the data
scaler = MinMaxScaler(feature_range=(0, 1))
closing_price = scaler.fit_transform(np.array(closing_price).reshape(-1, 1))

In [ ]:
# Create sequences for the transformer model
def create_sequences(dataset, time_step):
    dataX, dataY = [], []
    for i in range(len(dataset) - time_step):
        a = dataset[i:(i + time_step), 0]
        dataX.append(a)
        dataY.append(dataset[i + time_step, 0])
    return np.array(dataX), np.array(dataY)

# Time step for creating input sequences
time_step = 100

In [ ]:
# Input sequences and corresponding labels
X_transformer, y_transformer = create_sequences(closing_price, time_step)

In [ ]:
# Reshape the data to fit the transformer model
X_transformer = X_transformer.reshape(X_transformer.shape[0], X_transformer.shape[1], 1)

In [ ]:
# Model Building

# Transformer encoder block for the model
def transformer_encoder(inputs, head_size, num_heads, ff_dim, dropout=0):
    # Normalization and Attention
    x = layers.LayerNormalization(epsilon=1e-6)(inputs)
    x = layers.MultiHeadAttention(
        key_dim=head_size, num_heads=num_heads, dropout=dropout
    )(x, x)
    x = layers.Dropout(dropout)(x)
    res = x + inputs

    # Feed Forward Part
    x = layers.LayerNormalization(epsilon=1e-6)(res)
    x = layers.Conv1D(filters=ff_dim, kernel_size=1, activation="relu")(x)
    x = layers.Dropout(dropout)(x)
    x = layers.Conv1D(filters=inputs.shape[-1], kernel_size=1)(x)
    return x + res

In [ ]:
# Main function to build the entire model
def build_model(
    input_shape,
    head_size,
    num_heads,
    ff_dim,
    num_transformer_blocks,
    mlp_units,
    dropout=0,
    mlp_dropout=0,
):
    inputs = keras.Input(shape=input_shape)
    x = inputs
    for _ in range(num_transformer_blocks):
        x = transformer_encoder(x, head_size, num_heads, ff_dim, dropout)

    x = layers.GlobalAveragePooling1D(data_format="channels_first")(x)
    for dim in mlp_units:
        x = layers.Dense(dim, activation="relu")(x)
        x = layers.Dropout(mlp_dropout)(x)
    outputs = layers.Dense(1, activation="linear")(x)  # Linear activation for regression
    return keras.Model(inputs, outputs)

In [ ]:
# Get the input shape from the preprocessed data
input_shape = X_transformer.shape[1:]

# Model building with specified hyperparameters
model = build_model(
    input_shape,
    head_size=256,
    num_heads=4,
    ff_dim=4,
    num_transformer_blocks=4,
    mlp_units=[128],
    mlp_dropout=0.4,
    dropout=0.25,
)

In [ ]:
# Model Compiling with mean squared error loss and Adam optimizer
model.compile(
    loss="mean_squared_error",
    optimizer=keras.optimizers.Adam(learning_rate=1e-4),
)

# Display the summary of the model architecture
model.summary()

In [ ]:
# Early stopping to prevent overfitting
callbacks = [keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True)]

# Train the model on the input sequences and labels
model.fit(
    X_transformer,
    y_transformer,
    validation_split=0.2,
    epochs=5,
    batch_size=64,
    callbacks=callbacks,
)


# **Making Predictions**

In [ ]:
# Generate Input Sequence
input_sequence = X_transformer[-1]  # Last available sequence in your dataset

# Predictions
predictions = []
for _ in range(30):
    # Reshape input_sequence for prediction
    input_sequence_reshaped = input_sequence.reshape(1, input_sequence.shape[0], 1)

    # Make prediction
    prediction = model.predict(input_sequence_reshaped)

    # Append prediction to the list
    predictions.append(prediction[0, 0])

    # Update input_sequence for the next prediction
    input_sequence = np.roll(input_sequence, -1)
    input_sequence[-1] = prediction[0, 0]

# Inverse Transform
predictions = np.array(predictions).reshape(-1, 1)
predictions = scaler.inverse_transform(predictions)

# Plotting
plt.figure(figsize=(10, 6))
plt.plot(df['Close'], label='Actual Data')
plt.plot(np.arange(len(df['Close']), len(df['Close']) + 30), predictions, label='Predicted Data')
plt.title('Price Prediction for the Next 30 Days')
plt.xlabel('Date')
plt.ylabel('Closing Price')
plt.legend()
plt.show()


# **Actual vs Predicted Price**

In [ ]:
# Model Training
callbacks = [keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True)]

history = model.fit(
    X_transformer,
    y_transformer,
    validation_split=0.2,
    epochs=2,
    batch_size=64,
    callbacks=callbacks,
)

# Make predictions on the validation set
y_pred = model.predict(X_transformer)

# Inverse transform the normalized values to get actual closing prices
y_actual = scaler.inverse_transform(y_transformer.reshape(-1, 1))
y_pred_actual = scaler.inverse_transform(y_pred.reshape(-1, 1))

# Create a DataFrame for comparison
comparison_df = pd.DataFrame({
    'Actual': y_actual.flatten(),
    'Predicted': y_pred_actual.flatten()
})


In [ ]:
# Making predictions on the validation set
y_pred = model.predict(X_transformer)

# Inverse transform the normalized values to get actual closing prices
y_actual = scaler.inverse_transform(y_transformer.reshape(-1, 1))
y_pred_actual = scaler.inverse_transform(y_pred.reshape(-1, 1))

# Plotting the Actual vs Predicted closing prices
plt.figure(figsize=(10, 6))
plt.plot(y_actual, label='Actual Closing Price', color='lime')
plt.plot(y_pred_actual, label='Predicted Closing Price', color='red')
plt.title('Actual vs Predicted Closing Prices')
plt.xlabel('Time')
plt.ylabel('Closing Price')
plt.legend()
plt.show()